In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import(
 SGDRegressor,
 SGDClassifier,
 LinearRegression
)

from sklearn.ensemble import(
    AdaBoostRegressor,
    BaggingRegressor
)

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import (
    train_test_split,
    KFold,
    StratifiedKFold,
    GroupKFold,
    RepeatedStratifiedKFold,
    cross_val_score,
    permutation_test_score,
    GroupShuffleSplit,
)

from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance

from sklearn.model_selection import GridSearchCV
from mlxtend.feature_selection import SequentialFeatureSelector
from scipy.stats import sem
import os


In [43]:
#here we load the different datasets from the folder data/lexicon_analysi
df = pd.read_csv('Data/lexicon_analysis/word_diff_emotion_raw_scores.csv')

In [44]:
df

,Unnamed: 0,Word,Group,proportion_diff_depr,proportion_diff_anx,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,1,accident,7,-0.019615,0.005814,0,0,0,1,0,1,0,1,1,0
1,2,accordion,4,-0.094475,-0.086865,0,0,0,0,0,0,0,0,0,0
2,3,actor,8,-0.068694,-0.081373,0,0,0,0,0,0,0,0,0,0
3,4,advantage,1,0.001539,0.036491,0,0,0,0,0,0,1,0,0,0
4,5,advice,7,-0.074927,-0.020681,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,383,worth,3,-0.012449,-0.039607,0,0,0,0,0,0,1,0,0,0
383,384,year,2,-0.039544,-0.004483,0,0,0,0,0,0,0,0,0,0
384,385,yellow,5,0.008325,0.046420,0,0,0,0,0,0,0,0,0,0
385,386,zone,2,0.018780,-0.000138,0,0,0,0,0,0,0,0,0,0


# Holdout Set

In [45]:
#Here we choose which features to use
#here we make a train and holdout set. We keep groups seperated in training/holdout
#so there is no data leakage from the groups

#datasets for features and targets
X = df.iloc[:, 5:15].drop(['negative','positive'], axis=1)
y = df[['Group','proportion_diff_depr', 'proportion_diff_anx']]
groups = df[['Group']]
#print(groups)

gss = GroupShuffleSplit(n_splits = 2, train_size = .7, random_state = 42)
    
#train/holdout-split stratified by target
for train_idx, holdout_idx in gss.split(X, y, groups):
    X_train = X.iloc[train_idx, :]
    y_train = y.iloc[train_idx, :]
    X_holdout = X.iloc[holdout_idx, :]
    y_holdout = y.iloc[holdout_idx, :]

print(X_train, X_holdout, y_train, y_holdout)

     anger  disgust  fear  joy  sadness  surprise  trust
0        0        0     1    0        1         1      0
4        0        0     0    0        0         0      1
5        1        0     1    0        0         0      0
7        0        0     0    0        0         0      0
8        0        0     0    0        0         0      0
..     ...      ...   ...  ...      ...       ...    ...
379      0        0     0    0        0         0      1
382      0        0     0    0        0         0      0
383      0        0     0    0        0         0      0
384      0        0     0    0        0         0      0
385      0        0     0    0        0         0      0

[242 rows x 7 columns]      anger  disgust  fear  joy  sadness  surprise  trust
1        0        0     0    0        0         0      0
2        0        0     0    0        0         0      0
3        0        0     0    0        0         0      0
6        0        0     0    0        0         0      1
15     

In [58]:
#Here we save the splits so we can plot them in R later
#X_train.to_csv('Data/lexicon_analysis_performance/VAD_raw_scores_X_train.csv')
#X_holdout.to_csv('Data/lexicon_analysis_performance/VAD_raw_scores_X_holdout.csv')
#y_train.to_csv('Data/lexicon_analysis_performance/VAD_raw_scores_y_train.csv')
#y_holdout.to_csv('Data/lexicon_analysis_performance/VAD_raw_scores_y_holdout.csv')

# Regression Modelling

In [46]:
#defining function for permutation test for feature importance
def permutation_importance_test(features,target):
    r = permutation_importance(best_model,
                               features,
                               target,
                               n_repeats=100,
                               scoring = 'r2',
                               n_jobs=-1,
                               random_state=42)
    return(r.importances_mean, r.importances_std, r.importances)

In [47]:
#defining empty pandas dataframe
dataframe = pd.DataFrame(columns = ['target',
                                    'feature_names',
                                    'baseline_score',
                                    'baseline_std_dev',
                                    'baseline_std_error',
                                    'penalty_before_fs',
                                    'regurilization_strength_before_fs',
                                    'l1ratio_before_fs',
                                    'penalty_score_before_fs',
                                    'penalty_std_dev_before_fs',
                                    'fs_score',
                                    'fs_std_dev',
                                    'fs_std_error',
                                    'penalty_after_fs',
                                    'regurilization_strength_after_fs',
                                    'l1ratio_after_fs',
                                    'penalty_score_after_fs',
                                    'penalty_std_dev_after_fs',
                                    'boost_score',
                                    'boost_std_dev',
                                    'boost_std_error',
                                    'bag_score',
                                    'bag_std_dev',
                                    'bag_std_error',
                                    'earlystop_score',
                                    'earlystop_std_dev',
                                    'earlystop_std_error',
                                    'best_model_name',
                                    'best_model_train_r2_cv',
                                    'best_model_train_r2',
                                    'best_model_holdout_r2',
                                    'best_model_coef',
                                    'best_model_intercept',
                                    'predicted_values',
                                    'FI_mean_train',
                                    'FI_std_train',
                                    'FI_mean_holdout',
                                    'FI_std_holdout'])

#setting up variables for loop and appending
columns = list(dataframe)
data = []

groups_train = y_train['Group']
groups_holdout = y_holdout['Group']
cv = GroupKFold(n_splits=5)

#LOOP
#loop over targets

#looping over targets
for target in (range(1,3)):
    y_train_target = y_train.iloc[:, target]
    y_name = y_train_target.name
    y_holdout_target = y_holdout.iloc[:, target]

    print('******Start Iteration for Target %s*******' %(y_name))


    #-------------------------Baseline Model-----------------------------

    #Creating baseline model without any penalty
    baseline = LinearRegression(n_jobs = -1)

    #Evaluate by cv
    baseline_cv = cross_val_score(baseline,
                               X_train,
                               y_train_target,
                               scoring = 'r2', 
                               cv=cv, 
                               groups = groups_train,
                               n_jobs=-1,
                               error_score='raise')

    #Logging Scores
    baseline_score = baseline_cv.mean()
    baseline_std_dev = baseline_cv.std()
    baseline_std_error = sem(baseline_cv)

    print('Baseline Model Completed for target %s' %(y_name)) 


    #-------------------GRIDSEARCH L1/L2/Elasticnet--------------------------
    #-------------------(Before feature selection)-------------------------
    #-------------------Another way of feature selection-----------------------

    #Standard Scaling 
    #We fit the scaler only on the training data and standardise
    #both training and test features (not target!) with that scaler
    sc = StandardScaler()
    X_train_sc = sc.fit_transform(X_train)
    X_holdout_sc = sc.transform(X_holdout)

    # Defining penalties and value of inverse of regurilization strength
    # smaller values of C specify stronger regurilization.
    #also define l1ratio in case elasticnet is chosen 
    penalty = ['l1', 'l2', 'elasticnet', 'none']
    alpha = np.logspace(-3,3,7)
    l1_ratio = np.linspace(0,1,11)

    #defining a dictionary parameters to search
    parameters = [{"alpha":np.logspace(-3,3,7), 
                   "penalty": penalty,
                   'l1_ratio': l1_ratio}]

    #making the gridsearch and fitting features and target
    kep = GridSearchCV(
        estimator = SGDRegressor(random_state=42),
                    param_grid = parameters,
                    n_jobs = -1,
                    scoring = 'r2', 
                    cv=cv,
                    refit = True).fit(X_train_sc, y_train_target, groups = groups_train)

    #logging penalty, regurilization strength and best score
    penalty_before_fs = kep.best_estimator_.get_params()['penalty']
    regurilization_strength_before_fs = kep.best_estimator_.get_params()['alpha']
    l1ratio_before_fs = kep.best_estimator_.get_params()['l1_ratio']
    penalty_score_before_fs = kep.best_score_
    penalty_std_dev_before_fs = kep.cv_results_['std_test_score'][kep.best_index_]
    gridsearch_best_model_before_fs = kep.best_estimator_

    print('Gridsearch before Feature Selection Completed for target %s' %(y_name)) 

    #-------------------------Feature Selection-----------------------
    #making base estimator logistic regression with no penalty
    base = LinearRegression(n_jobs = -1)

    #making the forward selection with cv
    forward = SequentialFeatureSelector(
    base,
    k_features = 'best',
    cv = 5,
    forward = True, 
    floating = False,
    scoring = 'r2', 
    verbose = 2, 
    n_jobs = -1 
    ).fit(X_train, y_train_target)

    #saving forward-output to a dataframe
    le = pd.DataFrame.from_dict(forward.get_metric_dict()).T

    #finding the index of the best combination of features
    max_index = np.argmax(le.avg_score)

    #saving metrics from the best combination
    feature_names = le.iloc[max_index,3]
    fs_score = le.iloc[max_index,2]
    fs_std_dev = le.iloc[max_index, 5]
    fs_std_error = le.iloc[max_index, 6]

    #Choosing best features in X_train and use that in the rest of the loop
    feature_names_list = list(feature_names)
    X_train_after_fs = X_train[[c for c in X_train.columns if c in feature_names_list]]
    #defining holdout sets after feature selection
    X_holdout_after_fs = X_holdout[[c for c in X_holdout.columns if c in feature_names_list]]

    print('Feature Selection Completed for target %s' %(y_name))

    #------------------GRIDSEARCH L1/L2/Elasticnet-----------------------
    #------------------(After feature selection)-------------------------

    #Standard Scaling 
    #We fit the scaler only on the training data and standardise
    #both training and test features (not target!) with that scaler
    sc = StandardScaler()
    X_train_after_fs_sc = sc.fit_transform(X_train_after_fs)
    X_holdout_after_fs_sc = sc.transform(X_holdout_after_fs)

    # Defining penalties and value of inverse of regurilization strength
    # smaller values of C specify stronger regurilization.
    #also define l1ratio in case elasticnet is chosen 
    penalty = ['l1', 'l2', 'elasticnet', 'none']
    alpha = np.logspace(-3,3,7)
    l1_ratio = np.linspace(0,1,11)

    #defining a dictionary parameters to search
    parameters = [{"alpha":np.logspace(-3,3,7), 
                   "penalty": penalty,
                   'l1_ratio': l1_ratio}]

    #making the gridsearch and fitting features and target
    kep = GridSearchCV(
        estimator = SGDRegressor(random_state=42),
                    param_grid = parameters,
                    n_jobs = -1,
                    scoring = 'r2', 
                    cv=cv,
                    refit = True).fit(X_train_after_fs_sc, y_train_target, groups = groups_train)

    #logging penalty, regurilization strength and best score
    penalty_after_fs = kep.best_estimator_.get_params()['penalty']
    regurilization_strength_after_fs = kep.best_estimator_.get_params()['alpha']
    l1ratio_after_fs = kep.best_estimator_.get_params()['l1_ratio']
    penalty_score_after_fs = kep.best_score_
    penalty_std_dev_after_fs = kep.cv_results_['std_test_score'][kep.best_index_]
    gridsearch_best_model_after_fs = kep.best_estimator_

    print('Gridsearch after Feature Selection Completed for target %s' %(y_name))

    #-----------------------Adaboost and Bagging------------------------------
    #These three regurilization techniques apply the penalty selected
    #in the gridsearch above

    #AdaBoost
    boost = AdaBoostRegressor(base_estimator = gridsearch_best_model_after_fs,
                                      random_state=42)

    #Bagging
    bag = BaggingRegressor(base_estimator = gridsearch_best_model_after_fs,
                                     random_state=42,
                                     n_jobs = -1)

    #Evaluate Adaboost by cv
    boost_cv = cross_val_score(boost,
                               X_train_after_fs_sc,
                               y_train_target,
                               scoring = 'r2',
                               cv=cv, 
                               groups = groups_train,
                               n_jobs=-1,
                               error_score='raise')

    #Evaluate bagging by cv
    bag_cv = cross_val_score(bag,
                               X_train_after_fs_sc,
                               y_train_target,
                               scoring = 'r2',
                               cv=cv, 
                               groups = groups_train,
                               n_jobs=-1,
                               error_score='raise')

    #logging scores
    boost_score = boost_cv.mean()
    boost_std_dev = boost_cv.std()
    boost_std_error = sem(boost_cv)

    bag_score = bag_cv.mean()
    bag_std_dev = bag_cv.std()
    bag_std_error = sem(bag_cv)

    print('Adaboost and Bagging Completed for target %s' %(y_name))


    #----------------------------Earlystopping---------------------

    #base model for earlystop
    earlystop =  SGDRegressor( penalty=str(penalty_after_fs), 
                               alpha=regurilization_strength_after_fs,
                               l1_ratio=l1ratio_after_fs,
                               verbose=1,
                               early_stopping=True,
                               validation_fraction=0.2)

    #Evaluate earlystop by cv
    earlystop_cv = cross_val_score(earlystop,
                               X_train_after_fs_sc,
                               y_train_target,
                               scoring= 'r2',
                               cv=cv, 
                               groups = groups_train,
                               n_jobs=-1,
                               error_score='raise')


    earlystop_score = earlystop_cv.mean()
    earlystop_std_dev = earlystop_cv.std()
    earlystop_std_error = sem(earlystop_cv)

    print('Earlystop Completed for target %s' %(y_name))

    #-----------------------------Model Evaluation---------------------------

    #selecting the best model and its name
    scorings = (baseline_score, penalty_score_before_fs, fs_score,
              penalty_score_after_fs, boost_score, bag_score, earlystop_score)
    model_names = ('baseline_model', 'penalty_model_before_fs', 'feature_selection_model',
                   'penalty_model_after_fs', 'boost_model', 'bag_model', 'earlystop_model')
    index_max = scorings. index(max(scorings)) 
    best_model_name = model_names[index_max]
    best_model_train_r2_cv = scorings[index_max]
    print(best_model_name)

    #Defining the best model
    if index_max == 0:
        best_model = LinearRegression(n_jobs = -1)

    if index_max == 1:
        best_model = gridsearchbest_model_before_fs                                             

    elif index_max == 2:
        best_model = LinearRegression(n_jobs = -1)

    elif index_max == 3:
        best_model = gridsearch_best_model_after_fs

    elif index_max == 4:
        best_model = AdaBoostRegressor(base_estimator = gridsearch_best_model_after_fs,
                                      random_state=42)

    elif index_max == 5:
        best_model = BaggingRegressor(base_estimator = gridsearch_best_model_after_fs,
                                     random_state=42,
                                     n_jobs = -1)
    elif index_max == 6:
        best_model =  SGDRegressor(penalty=str(penalty_after_fs), 
                                   alpha=regurilization_strength_after_fs,
                                   l1_ratio=l1ratio_after_fs,
                                   verbose=1,
                                   early_stopping=True,
                                   validation_fraction=0.2)

   #Get fit, score, coefficients, intercept and apply permutation test on
   #the best model based on whether it is before or after feature selection or with scaled features

    #using features before feature selection (non-scaled)
    if index_max == 0:
        best_model = best_model.fit(X_train, y_train_target)
        best_model_train_r2 = best_model.score(X_train, y_train_target)
        best_model_holdout_r2 = best_model.score(X_holdout, y_holdout_target)
        predicted_values = best_model.predict(X_holdout)
        
        #permutation feature importance for train and holdout
        FI_mean_train, FI_std_train, FI_importances_train = permutation_importance_test(
        X_train, y_train_target)
        FI_mean_holdout, FI_std_holdout, FI_importances_holdout = permutation_importance_test(
        X_holdout, y_holdout_target)

        
    #using features before feature selection and scaled
    elif index_max == 1:
        best_model = best_model.fit(X_train_sc, y_train_target)
        best_model_train_r2 = best_model.score(X_train_sc, y_train_target)
        best_model_holdout_r2 = best_model.score(X_holdout_sc, y_holdout_target)
        predicted_values = best_model.predict(X_holdout_sc)
        
        #permutation feature importance for train and holdout
        FI_mean_train, FI_std_train, FI_importances_train = permutation_importance_test(
        X_train_sc, y_train_target)
        FI_mean_holdout, FI_std_holdout, FI_importances_holdout = permutation_importance_test(
        X_holdout_sc, y_holdout_target)


    #using features after feature selection (non-scaled)
    elif index_max == 2:
        best_model = best_model.fit(X_train_after_fs, y_train_target)
        best_model_train_r2 = best_model.score(X_train_after_fs, y_train_target)
        best_model_holdout_r2 = best_model.score(X_holdout_after_fs, y_holdout_target)
        predicted_values = best_model.predict(X_holdout_after_fs)
        
        #permutation feature importance for train and holdout
        FI_mean_train, FI_std_train, FI_importances_train = permutation_importance_test(
        X_train_after_fs, y_train_target)
        FI_mean_holdout, FI_std_holdout, FI_importances_holdout = permutation_importance_test(
        X_holdout_after_fs, y_holdout_target)


    #using features after feature selection and scaled   
    elif index_max in (3,4,5,6):
        best_model = best_model.fit(X_train_after_fs_sc, y_train_target)
        best_model_train_r2 = best_model.score(X_train_after_fs_sc, y_train_target)
        best_model_holdout_r2 = best_model.score(X_holdout_after_fs_sc, y_holdout_target)
        predicted_values = best_model.predict(X_holdout_after_fs_sc)
        
        #permutation feature importance for train and holdout
        FI_mean_train, FI_std_train, FI_importances_train = permutation_importance_test(
        X_train_after_fs_sc, y_train_target)
        FI_mean_holdout, FI_std_holdout, FI_importances_holdout = permutation_importance_test(
        X_holdout_after_fs_sc, y_holdout_target)
        
    best_model_coef = best_model.coef_
    best_model_intercept = best_model.intercept_

    print('Fitting Best Model on Holdout Set Completed target %s' %(y_name))


   #--------------------------Saving output-----------------------------

    #defining outputs as list
    output = [y_name,
              feature_names,
              baseline_score,
              baseline_std_dev,
              baseline_std_error,
              penalty_before_fs,
              regurilization_strength_before_fs,
              l1ratio_before_fs,
              penalty_score_before_fs,
              penalty_std_dev_before_fs,
              fs_score,
              fs_std_dev,
              fs_std_error,
              penalty_after_fs,
              regurilization_strength_after_fs,
              l1ratio_after_fs,
              penalty_score_after_fs,
              penalty_std_dev_after_fs,
              boost_score,
              boost_std_dev,
              boost_std_error,
              bag_score,
              bag_std_dev,
              bag_std_error,
              earlystop_score,
              earlystop_std_dev,
              earlystop_std_error,
              best_model_name,
              best_model_train_r2_cv,
              best_model_train_r2,
              best_model_holdout_r2,
              best_model_coef,
              best_model_intercept,
              predicted_values,
              FI_mean_train,
              FI_std_train,
              FI_mean_holdout,
              FI_std_holdout]

    #turning outputs into zipped
    zipped = zip(columns, output)
    #turning zipped into dictionary
    output_dict = dict(zipped)
    #appending to our data
    data.append(output_dict)


#finally (phew) appending data to dataframe
dataframe = dataframe.append(data, ignore_index = True)

******Start Iteration for Target proportion_diff_depr*******
Baseline Model Completed for target proportion_diff_depr
Gridsearch before Feature Selection Completed for target proportion_diff_depr


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished

[2022-10-17 16:18:05] Features: 1/7 -- score: 0.01849533694229888[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.0s finished

[2022-10-17 16:18:05] Features: 2/7 -- score: 0.012195244497252577[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished

[2022-10-17 16:18:05] Features: 3/7 -- score: -0.0009228548126353431[Parallel(n_jobs=-1)]: Using back

Feature Selection Completed for target proportion_diff_depr
Gridsearch after Feature Selection Completed for target proportion_diff_depr
Adaboost and Bagging Completed for target proportion_diff_depr
Earlystop Completed for target proportion_diff_depr
feature_selection_model
Fitting Best Model on Holdout Set Completed target proportion_diff_depr
******Start Iteration for Target proportion_diff_anx*******
Baseline Model Completed for target proportion_diff_anx
Gridsearch before Feature Selection Completed for target proportion_diff_anx


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.0s finished

[2022-10-17 16:18:09] Features: 1/7 -- score: 0.03222354686784048[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.0s finished

[2022-10-17 16:18:09] Features: 2/7 -- score: 0.04384159939343033[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished

[2022-10-17 16:18:09] Features: 3/7 -- score: 0.04140314070689919[Parallel(n_jobs=-1)]: Using backend 

Feature Selection Completed for target proportion_diff_anx
Gridsearch after Feature Selection Completed for target proportion_diff_anx
Adaboost and Bagging Completed for target proportion_diff_anx
Earlystop Completed for target proportion_diff_anx
feature_selection_model
Fitting Best Model on Holdout Set Completed target proportion_diff_anx



[2022-10-17 16:18:10] Features: 7/7 -- score: -0.0007369389762936329

In [41]:
#dataframe.to_csv('Data/lexicon_analysis_performance/emotion_raw_scores_negposremoved.csv')

In [48]:
dataframe

,target,feature_names,baseline_score,baseline_std_dev,baseline_std_error,penalty_before_fs,regurilization_strength_before_fs,l1ratio_before_fs,penalty_score_before_fs,penalty_std_dev_before_fs,...,best_model_train_r2_cv,best_model_train_r2,best_model_holdout_r2,best_model_coef,best_model_intercept,predicted_values,FI_mean_train,FI_std_train,FI_mean_holdout,FI_std_holdout
0,proportion_diff_depr,"(fear,)",-0.190664,0.248601,0.124300,l2,1.00,0.0,-0.158259,0.241866,...,0.018495,0.040898,0.015701,[-0.03456029007539063],-0.001969,"[-0.0019692955104200103, -0.001969295510420010...",[0.08137222440482281],[0.028039688695025527],[0.08014152049262334],[0.02813979265417173]
1,proportion_diff_anx,"(disgust, sadness)",-0.124430,0.210671,0.105336,elasticnet,0.01,0.2,-0.095050,0.174047,...,0.043842,0.086096,0.086744,"[-0.04375844738884718, -0.036668547786558356]",0.003228,"[0.0032277562033045572, 0.0032277562033045572,...","[0.06074508029805657, 0.05220358844218828]","[0.020374982585213128, 0.02087310630175312]","[0.05895326952970903, 0.05195713957306466]","[0.02277187167354429, 0.022421899551549135]"
